<a href="https://colab.research.google.com/github/savagecm/codeJam/blob/master/max.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from __future__ import print_function
# importing required modules 
from zipfile import ZipFile 
import requests
import numpy as np
import pandas as pd
import os
from imblearn.over_sampling import SMOTE, ADASYN

# specifying the zip file name 
train_file_name = "orange_small_train.data.zip"
test_file_name = "orange_small_test.data.zip"
trainAppetencyName = "orange_small_train_appetency.labels"
trainChrunName = "orange_small_train_churn.labels"
trainUpselling = "orange_small_train_upselling.labels"


def downloadFile(fileName):
  urlBase = "https://www.kdd.org/cupfiles/KDDCupData/2009/"
  urlName = urlBase+fileName
  print("download file from URL : ", urlName)
  response = requests.get(urlName, stream=True)
  response.status_code
  with open(fileName, "wb") as code:
    code.write(response.content)
  return

def downloadZipFile(fileName):
  urlBase = "https://www.kdd.org/cupfiles/KDDCupData/2009/"
  urlName = urlBase+fileName
  print("download file from URL : ", urlName)
  response = requests.get(urlName, stream=True)
  response.status_code
  with open(fileName, "wb") as code:
    code.write(response.content)
  # opening the zip file in READ mode 
  with ZipFile(fileName, 'r') as zip: 
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 
  return





downloadZipFile(train_file_name);
downloadZipFile(test_file_name);
downloadFile(trainAppetencyName);
downloadFile(trainChrunName);
downloadFile(trainUpselling);


print(os.listdir(os.getcwd()))

download file from URL :  https://www.kdd.org/cupfiles/KDDCupData/2009/orange_small_train.data.zip
Extracting all the files now...
Done!
download file from URL :  https://www.kdd.org/cupfiles/KDDCupData/2009/orange_small_test.data.zip
Extracting all the files now...
Done!
download file from URL :  https://www.kdd.org/cupfiles/KDDCupData/2009/orange_small_train_appetency.labels
download file from URL :  https://www.kdd.org/cupfiles/KDDCupData/2009/orange_small_train_churn.labels
download file from URL :  https://www.kdd.org/cupfiles/KDDCupData/2009/orange_small_train_upselling.labels
['.config', 'orange_small_train_upselling.labels', 'orange_small_train_churn.labels', 'orange_small_train.data', 'orange_small_train_appetency.labels', 'orange_small_train.data.cksum', 'orange_small_test.data', 'orange_small_test.data.cksum', 'orange_small_test.data.zip', 'orange_small_train.data.zip', 'sample_data']


read csv data

In [4]:
train = pd.read_csv ('orange_small_train.data',sep ='\t')
test = pd.read_csv ('orange_small_test.data',sep ='\t')
appetency = pd.read_table('orange_small_train_appetency.labels', header = None).loc[:, 0].astype('category')
churn = pd.read_table('orange_small_train_churn.labels', header = None).loc[:, 0].astype('category')
upselling= pd.read_table('orange_small_train_upselling.labels', header = None).loc[:, 0].astype('category')
train.describe()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var153,Var154,Var155,Var156,Var157,Var158,Var159,Var160,Var161,Var162,Var163,Var164,Var165,Var166,Var167,Var168,Var169,Var170,Var171,Var172,Var173,Var174,Var175,Var176,Var177,Var178,Var179,Var180,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190,Var209,Var230
count,702.000000,1241.000000,1240.000000,1579.000000,1.487000e+03,44471.000000,44461.000000,0.0,702.000000,1.487000e+03,1240.000000,558.000000,44461.000000,1240.000000,0.0,1487.000000,1579.000000,1579.000000,1579.000000,0.0,44471.000000,44991.000000,1487.000000,42770.000000,44991.000000,1487.000000,1487.000000,44989.000000,702.000000,702.000000,0.0,0.0,8.470000e+02,1241.000000,44991.000000,1.241000e+03,1.579000e+03,4.499100e+04,0.0,1241.000000,...,4.499100e+04,7.020000e+02,1579.00000,694.000000,1129.000000,873.000000,1241.000000,44991.000000,1579.000000,1.241000e+03,4.499100e+04,1579.000000,8.730000e+02,1487.000000,0.0,702.000000,0.0,1241.000000,1.083000e+03,1487.000000,44991.000000,1579.000000,0.0,1240.000000,1.241000e+03,646.000000,1579.000000,7.020000e+02,44991.000000,1.579000e+03,1.241000e+03,1241.000000,0.0,702.000000,702.00000,1241.000000,21022.000000,333.000000,0.0,0.0
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,NaN,48.145299,3.926057e+05,8.625806,16.071685,1249.688401,0.741935,NaN,120.239275,11.393287,7.215959,0.245092,NaN,234.518225,290.245382,7.535306,4.507926,96.827010,0.070612,0.028245,224.507669,0.022792,7.435897,NaN,NaN,1.270050e+05,1.173247,0.716810,1.595539e+05,6.485221e+05,2.579107e+06,NaN,13.962933,...,6.181967e+06,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.713940,38.803005,3.374288,3.360168e+05,4.860780e+05,1.753642,2.884236e+04,22.553463,NaN,332.938575,NaN,3.041096,3.674517e+05,9.744452,0.006846,7.044965,NaN,4.716129,6.188884e+05,16.687307,3.138062,3.776755e+06,0.611456,1.416638e+06,7.777380e+04,8.460919,NaN,3.299145,16.54416,167.368477,270.142137,22007.045192,NaN,NaN
std,40.709951,0.141933,4270.193518,1.275481,6.441259e+05,2685.693668,6.326053,NaN,154.777855,9.280896e+05,2.869558,64.185508,2794.954874,3.714316,NaN,72.386254,49.493856,34.415427,1.781497,NaN,565.560147,704.489990,49.449472,9.928819,214.318283,0.528836,0.247210,98.520240,0.212436,8.852461,NaN,NaN,4.957921e+05,4.377707,2.996007,3.277151e+05,1.382225e+06,3.010076e+06,NaN,81.410005,...,4.348926e+06,2.280224e+06,3.04295,544.686538,187.525494,6.325709,11.027292,99.497149,8.579672,9.731987e+05,8.488638e+05,8.104872,1.157869e+05,96.211261,NaN,102.879214,NaN,27.921735,6.042377e+05,10.397319,0.132503,36.186111,NaN,31.002253,1.306030e+06,59.016629,28.206035,3.785696e+06,2.495681,2.279786e+06,2.016188e+05,46.973777,NaN,8.781967,60.22303,113.980072,86.707692,29085.146490,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000e+00,8.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-66.880000,0.000000,0.000000,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000,...,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,NaN,0.000000,NaN,0.000000,0.000000e+00,0.000000,0.000000,0.000000,NaN,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN,0.000000,0.00000,-6.420000,6.000000,0.000000,NaN,NaN
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,518.000000,0.000000,NaN,4.000000,0.000000e+00,8.000000,0.000000,0.000000,0.000000,NaN,51.920000,0.000000,0.000000,0.000000,NaN,112.000000,135.000000,0.000000,0.000000,16.000000,0.000000,0.000000,166.560000,0.000000,0.000000,NaN,NaN,0.000000e+00,0.000000,0.0000

smote test

In [0]:
train.head(100)

In [0]:
DataVars = train.columns
data_types = {Var: train[Var].dtype for Var in DataVars}
data_types

In [7]:
from sklearn import preprocessing
from fancyimpute import KNN
train_cat = train.select_dtypes(exclude=[object])
train_cat = train_cat.fillna(train_cat.mean())


"""
# this is too slow
data_incomplete = np.array(train_cat)#转为array
        #预测缺失值
filled_knn = KNN(k=3).fit_transform(data_incomplete)#利用knn填补缺失值
data_complete = pd.DataFrame(filled_knn)#保存结果
data_complete.head()
"""

train_cat.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var153,Var154,Var155,Var156,Var157,Var158,Var159,Var160,Var161,Var162,Var163,Var164,Var165,Var166,Var167,Var168,Var169,Var170,Var171,Var172,Var173,Var174,Var175,Var176,Var177,Var178,Var179,Var180,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190,Var209,Var230
0,11.487179,0.004029,425.298387,0.125396,238793.32885,1526.000000,7.0,NaN,48.145299,392605.656355,8.625806,16.071685,184.0,0.741935,NaN,120.239275,11.393287,7.215959,0.245092,NaN,464.000000,580.0,7.535306,14.000000,128.0,0.070612,0.028245,166.56,0.022792,7.435897,NaN,NaN,127004.950413,1.173247,0.0,159553.853344,648522.148195,3570.0,NaN,13.962933,...,1812252.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,142.0,3.374288,336016.762288,38418.0,1.753642,28842.355097,22.553463,NaN,332.938575,NaN,3.041096,367451.723236,9.744452,0.0,7.044965,NaN,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,NaN,3.299145,16.54416,167.368477,462.000000,22007.045192,NaN,NaN
1,11.487179,0.004029,425.298387,0.125396,238793.32885,525.000000,0.0,NaN,48.145299,392605.656355,8.625806,16.071685,0.0,0.741935,NaN,120.239275,11.393287,7.215959,0.245092,NaN,168.000000,210.0,7.535306,2.000000,24.0,0.070612,0.028245,353.52,0.022792,7.435897,NaN,NaN,127004.950413,1.173247,0.0,159553.853344,648522.148195,4764966.0,NaN,13.962933,...,10439160.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,32.0,3.374288,336016.762288,238572.0,1.753642,28842.355097,22.553463,NaN,332.938575,NaN,3.041096,367451.723236,9.744452,0.0,7.044965,NaN,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,NaN,3.299145,16.54416,167.368477,270.142137,22007.045192,NaN,NaN
2,11.487179,0.004029,425.298387,0.125396,238793.32885,5236.000000,7.0,NaN,48.145299,392605.656355,8.625806,16.071685,904.0,0.741935,NaN,120.239275,11.393287,7.215959,0.245092,NaN,1212.000000,1515.0,7.535306,26.000000,816.0,0.070612,0.028245,220.08,0.022792,7.435897,NaN,NaN,127004.950413,1.173247,0.0,159553.853344,648522.148195,5883894.0,NaN,13.962933,...,9826360.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,206.0,3.374288,336016.762288,434946.0,1.753642,28842.355097,22.553463,NaN,332.938575,NaN,3.041096,367451.723236,9.744452,0.0,7.044965,NaN,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,NaN,3.299145,16.54416,167.368477,270.142137,22007.045192,NaN,NaN
3,11.487179,0.004029,425.298387,0.125396,238793.32885,1326.437116,0.0,NaN,48.145299,392605.656355,8.625806,16.071685,0.0,0.741935,NaN,120.239275,11.393287,7.215959,0.245092,NaN,234.518225,0.0,7.535306,4.507926,0.0,0.070612,0.028245,22.08,0.022792,7.435897,NaN,NaN,127004.950413,1.173247,0.0,159553.853344,648522.148195,0.0,NaN,13.962933,...,0.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,0.0,3.374288,336016.762288,0.0,1.753642,28842.355097,22.553463,NaN,332.938575,NaN,3.041096,367451.723236,9.744452,0.0,7.044965,NaN,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,NaN,3.299145,16.54416,167.368477,270.142137,22007.045192,NaN,NaN
4,11.487179,0.004029,425.298387,0.125396,238793.32885,1029.000000,7.0,NaN,48.145299,392605.656355,8.625806,16.071685,3216.0,0.741935,NaN,120.239275,11.393287,7.215959,0.245092,NaN,64.000000,80.0,7.535306,4.000000,64.0,0.070612,0.028245,200.00,0.022792,7.435897,NaN,NaN,127004.950413,1.173247,0.0,159553.853344,648522.148195,0.0,NaN,13.962933,...,644836.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,2.0,3.374288,336016.762288,0.0,1.753642,28842.355097,22.553463,NaN,332.938575,NaN,3.041096,367451.723236,9.744452,0.0,7.044965,NaN,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.46

In [11]:
threshold = 0.2

train_cat = train_cat.drop(train_cat.std()[train_cat.std() < threshold].index.values, axis=1)
train_cat = train_cat.loc[:, pd.notnull(train_cat).sum()>len(train_cat)*.8]
train_cat.head()

,Var1,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,Var12,Var13,Var14,Var16,Var17,Var18,Var19,Var21,Var22,Var23,Var24,Var25,Var28,Var30,Var33,Var34,Var35,Var36,Var37,Var38,Var40,Var41,Var43,Var44,Var45,Var46,Var47,Var50,Var51,Var53,Var54,...,Var146,Var147,Var148,Var149,Var150,Var151,Var152,Var153,Var154,Var155,Var156,Var157,Var158,Var159,Var160,Var161,Var162,Var163,Var164,Var165,Var166,Var168,Var170,Var171,Var172,Var174,Var176,Var177,Var178,Var179,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
0,11.487179,425.298387,0.125396,238793.32885,1526.000000,7.0,48.145299,392605.656355,8.625806,16.071685,184.0,0.741935,120.239275,11.393287,7.215959,0.245092,464.000000,580.0,7.535306,14.000000,128.0,166.56,7.435897,127004.950413,1.173247,0.0,159553.853344,648522.148195,3570.0,13.962933,26.653846,4.157937,0.0,7256.127907,16.796132,2.188034,35.690883,43652.006725,654326.5,4.789686,...,2.781439,1.701412,140.486886,397579.000000,157687.615579,11.060213,8.314123,1812252.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,142.0,3.374288,336016.762288,38418.0,1.753642,28842.355097,22.553463,332.938575,3.041096,367451.723236,9.744452,7.044965,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,462.000000,22007.045192
1,11.487179,425.298387,0.125396,238793.32885,525.000000,0.0,48.145299,392605.656355,8.625806,16.071685,0.0,0.741935,120.239275,11.393287,7.215959,0.245092,168.000000,210.0,7.535306,2.000000,24.0,353.52,7.435897,127004.950413,1.173247,0.0,159553.853344,648522.148195,4764966.0,13.962933,26.653846,4.157937,0.0,7256.127907,16.796132,2.188034,35.690883,43652.006725,654326.5,4.789686,...,2.781439,1.701412,140.486886,278334.000000,157687.615579,11.060213,8.314123,10439160.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,32.0,3.374288,336016.762288,238572.0,1.753642,28842.355097,22.553463,332.938575,3.041096,367451.723236,9.744452,7.044965,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
2,11.487179,425.298387,0.125396,238793.32885,5236.000000,7.0,48.145299,392605.656355,8.625806,16.071685,904.0,0.741935,120.239275,11.393287,7.215959,0.245092,1212.000000,1515.0,7.535306,26.000000,816.0,220.08,7.435897,127004.950413,1.173247,0.0,159553.853344,648522.148195,5883894.0,13.962933,26.653846,4.157937,0.0,7256.127907,16.796132,2.188034,35.690883,43652.006725,654326.5,4.789686,...,2.781439,1.701412,140.486886,320565.000000,157687.615579,11.060213,8.314123,9826360.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,206.0,3.374288,336016.762288,434946.0,1.753642,28842.355097,22.553463,332.938575,3.041096,367451.723236,9.744452,7.044965,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
3,11.487179,425.298387,0.125396,238793.32885,1326.437116,0.0,48.145299,392605.656355,8.625806,16.071685,0.0,0.741935,120.239275,11.393287,7.215959,0.245092,234.518225,0.0,7.535306,4.507926,0.0,22.08,7.435897,127004.950413,1.173247,0.0,159553.853344,648522.148195,0.0,13.962933,26.653846,4.157937,0.0,7256.127907,16.796132,2.188034,35.690883,43652.006725,654326.5,4.789686,...,2.781439,1.701412,140.486886,294920.804255,157687.615579,11.060213,8.314123,0.0,1.538221e+06,0.80114,169.472622,33.073516,1.896907,4.71394,0.0,3.374288,336016.762288,0.0,1.753642,28842.355097,22.553463,332.938575,3.041096,367451.723236,9.744452,7.044965,4.716129,618888.394037,16.687307,3.138062,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
4,11.487179,425.298387,0.125396,238793.32885,1029.000000,7.0,48.145299,392605.656355,8.625806,16.071685,3216.0,0.741935,120.239275,11.393287,7.215959,0.245092,64.000000,80.0,7.535306,4.000000,64.0,200.00,7.435897,127004.950413,1.173247,0.0,159553.853344,648522.148195,0.0,13.962933,26.653846,4.157937

In [18]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE(kind='svm').fit_sample(train_cat, appetency)
print(y_resampled)

[-1 -1 -1 ...  1  1  1]


In [21]:
print(y_resampled.shape)
print(train_cat.shape)

(75883,)


(50000, 160)



    Let's count how many rows have values

    And count how many columns have null value



In [0]:
train.apply(lambda x: x.count(), axis=1)

In [0]:
train.isnull().sum(axis=0)

    Let's start to clean data

    Drop the columns that std is close to be zero

    Delete columns with at least 20% missing values


In [0]:
threshold = 0.2

train = train.drop(train.std()[train.std() < threshold].index.values, axis=1)
train = train.loc[:, pd.notnull(train).sum()>len(train)*.8]



    We anly have 65 columns now.

    Let's change the data types:

    convert integer columns to float

    non-integer, non-float columns to categorical



In [0]:
DataVars = train.columns
data_types = {Var: train[Var].dtype for Var in DataVars}

for Var in DataVars:
    if data_types[Var] == int:
        x = train[Var].astype(float)
        train.loc[:, Var] = x
        data_types[Var] = x.dtype
    elif data_types[Var] != float:
        x = train[Var].astype('category')
        train.loc[:, Var] = x
        data_types[Var] = x.dtype
float_DataVars = [Var for Var in DataVars if data_types[Var] == float]
data_types

{'Var109': dtype('float64'),
 'Var112': dtype('float64'),
 'Var113': dtype('float64'),
 'Var119': dtype('float64'),
 'Var123': dtype('float64'),
 'Var125': dtype('float64'),
 'Var13': dtype('float64'),
 'Var132': dtype('float64'),
 'Var133': dtype('float64'),
 'Var134': dtype('float64'),
 'Var140': dtype('float64'),
 'Var143': dtype('float64'),
 'Var144': dtype('float64'),
 'Var149': dtype('float64'),
 'Var153': dtype('float64'),
 'Var160': dtype('float64'),
 'Var163': dtype('float64'),
 'Var181': dtype('float64'),
 'Var192': CategoricalDtype(categories=['0G9vyxdMWg', '0kQTmBU3gb', '0kQqrQsiZt', '0vimfo8zhV',
                   '1GdOj17ejg', '1GdOj1KXzC', '1JGTmBQZiT', '1JGqrQKzJV',
                   '1JGr4RKzJV', '1JGr4RUaLY',
                   ...
                   'yaKt0Gq076', 'ym_vyxTlru', 'ypNt0Go8Lb', 'zC_Wq6e2Rv',
                   'zKnq9at_88', 'zKnr4RXktW', 'zKnrjIPxRp', 'zcROj17IEC',
                   'zcROj1KVEH', 'zcRZptzip9'],
                  ordered=False),
 'Var1



Imputing the missing values for numerical variables: >

    Let’s fill up the missing values with the means of the respective columns:



In [0]:
# mark zero values as missing or NaN
#train[float_DataVars].isnull().replace(0, np.NaN)
# fill missing values with mean column values
#train[float_DataVars].fillna(train[float_DataVars].mean(), inplace=True)
# count the number of NaN values in each column
#train[float_DataVars]  

float_x_means = train.mean()

for Var in float_DataVars:
    x = train[Var]
    isThereMissing = x.isnull()
    if isThereMissing.sum() > 0:
        train.loc[isThereMissing.tolist(), Var] = float_x_means[Var]     



Cleaning Categorical Variables

    Now, time to fill the missing values of the categorical features



In [0]:
#Let's see the number of categories of each categorical feature:

DataVars = train.columns

categorical_DataVars = [Var for Var in DataVars
                           if data_types[Var] != float]

categorical_levels = train[categorical_DataVars].apply(lambda col: len(col.cat.categories))
categorical_levels


Var192      361
Var193       51
Var195       23
Var196        4
Var197      225
Var198     4291
Var199     5073
Var202     5713
Var203        5
Var204      100
Var205        3
Var206       21
Var207       14
Var208        2
Var210        6
Var211        2
Var212       81
Var216     2016
Var217    13990
Var218        2
Var219       22
Var220     4291
Var221        7
Var222     4291
Var223        4
Var226       23
Var227        7
Var228       30
dtype: int64

Those variables having over 500 categories are likely to be just text / character data. 
Let's get rid of them:

In [0]:
categorical_DataVars = categorical_levels[categorical_levels <= 500].index

categorical_DataVars

Index(['Var192', 'Var193', 'Var195', 'Var196', 'Var197', 'Var203', 'Var204',
       'Var205', 'Var206', 'Var207', 'Var208', 'Var210', 'Var211', 'Var212',
       'Var218', 'Var219', 'Var221', 'Var223', 'Var226', 'Var227', 'Var228'],
      dtype='object')



Let's create a new categorical feature named 'U' for "Unknown" for imputing the missing values for categorical variables. That will be a good enough practice, since we are not losing any samples and not making any assumptions on features distribution.


In [0]:
collapsed_categories = {}
removed_categorical_DataVars = set()

for Vars in categorical_DataVars:
    
    isTheremissing_value = train[Vars].isnull()
    if isTheremissing_value.sum() > 0:
        train[Vars].cat.add_categories('unknown', inplace=True)
        train.loc[isTheremissing_value.tolist(), Vars] = 'unknown'

In [0]:

train.head()



,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,Var44,Var57,Var65,Var73,Var74,Var76,Var78,Var81,Var83,Var85,Var109,Var112,Var113,Var119,Var123,Var125,Var132,Var133,Var134,Var140,Var143,Var144,Var149,Var153,Var160,Var163,Var181,Var192,Var193,Var195,Var196,Var197,Var198,Var199,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var210,Var211,Var212,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var226,Var227,Var228
0,1526.000000,7.0,184.0,464.000000,580.0,14.000000,128.0,166.56,0.0,3570.0,0.0,4.076907,9.0,36.0,35.0,1350864.0,0.0,7333.110000,5.0,12.0,104.00000,168.0,117625.60,1175.000000,6.0,720.0,0.0,1212385.0,69134.0,185.0,0.0,9.000000,397579.000000,1812252.0,142.0,38418.0,0.0,bZkvyxLkBI,RO12,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,dXGu,9_Y1,FbIm,VpdQ,haYg,me75fM6ugJ,kIsH,uKAI,L84s,XfqtO3UdzaXh_,XTbPUYD,sH5Z,cJvF,FzaX,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,xb3V,RAYp,F2FyR07IdsN7I
1,525.000000,0.0,0.0,168.000000,210.0,2.000000,24.0,353.52,0.0,4764966.0,0.0,5.408032,9.0,26.0,0.0,2872928.0,3.0,151098.900000,25.0,2.0,40.00000,40.0,-356411.60,590.000000,72.0,0.0,8.0,4136430.0,357038.0,0.0,0.0,9.000000,278334.000000,10439160.0,32.0,238572.0,0.0,CEat0G8rTN,RO12,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,lg1t,9_Y1,k13i,sJzTlal,zm5i,me75fM6ugJ,kIsH,uKAI,L84s,NhsEn4L,kZJyVg2,NaN,unknown,FzaX,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,fKCe,RAYp,F2FyR07IdsN7I
2,5236.000000,7.0,904.0,1212.000000,1515.0,26.000000,816.0,220.08,0.0,5883894.0,0.0,6.599658,9.0,130.0,518.0,1675776.0,0.0,16211.580000,40.0,58.0,312.00000,336.0,405104.00,3230.000000,114.0,5967.0,0.0,3478905.0,248932.0,800.0,0.0,36.000000,320565.000000,9826360.0,206.0,434946.0,0.0,eOQt0GoOh3,AERks4l,taul,1K8T,ffXs,NldASpP,y4g9XoZ,4bTR,9_Y1,MGOA,VpdQ,haYg,DHn_WUyBhW_whjA88g9bvA64_,kIsH,uKAI,L84s,UbxQ8lZ,pMWAe2U,bHR7,UYBR,FzaX,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,Qu4f,02N6s8f,ib5G6X1eUxUn6
3,1326.437116,0.0,0.0,234.518225,0.0,4.507926,0.0,22.08,0.0,0.0,0.0,1.988250,9.0,12.0,0.0,0.0,0.0,103084.052693,0.0,0.0,60.88866,0.0,-275703.60,916.112185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.727665,294920.804255,0.0,0.0,0.0,0.0,jg69tYsGvO,RO12,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,W8mQ,9_Y1,YULl,VpdQ,unknown,me75fM6ugJ,kIsH,uKAI,Mtgm,NhsEn4L,kq0dQfu,eKej,UYBR,FzaX,L91KIiz,oslk,CE7uk3u,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I
4,1029.000000,7.0,3216.0,64.000000,80.0,4.000000,64.0,200.00,0.0,0.0,0.0,4.552446,18.0,82.0,224.0,784448.0,0.0,37423.500000,0.0,0.0,32.00000,56.0,10714.84,215.000000,0.0,15111.0,0.0,150650.0,66046.0,3255.0,0.0,9.000000,267162.000000,644836.0,2.0,0.0,0.0,IXSgUHShse,RO12,taul,1K8T,uNkU,EKR938I,ThrHXVS,xklU,9_Y1,RVjC,sJzTlal,6JmL,me75fM6ugJ,kIsH,uKAI,L84s,XfqtO3UdzaXh_,11p4mKe,H3p7,UYBR,FzaX,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I


In [0]:
train.corr()

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,Var44,Var57,Var65,Var73,Var74,Var76,Var78,Var81,Var83,Var85,Var109,Var112,Var113,Var119,Var123,Var125,Var132,Var133,Var134,Var140,Var143,Var144,Var149,Var153,Var160,Var163,Var181
Var6,1.000000,0.148626,0.094397,0.576973,0.575632,0.419153,0.505610,-0.016302,-0.005624,0.021091,0.009223,-0.001842,0.102393,0.121403,0.023319,0.021117,-0.007073,-0.200376,0.489663,0.461425,0.496236,0.511474,0.015099,0.753037,0.517737,0.024524,-0.010569,0.023809,0.015377,0.057759,-0.006809,0.089661,0.001698,0.035678,0.502046,0.015913,-0.015613
Var7,0.148626,1.000000,0.406322,0.063691,0.067912,0.065608,0.080325,-0.022627,-0.014688,0.020216,-0.011820,-0.002905,0.650274,0.450135,0.126454,0.033393,0.008812,-0.139800,0.029021,0.077624,0.054886,0.078194,0.022057,0.071555,0.038178,0.085545,-0.019663,0.039435,0.035902,0.271089,-0.011605,0.314555,-0.002322,0.050166,0.063407,0.022137,-0.051640
Var13,0.094397,0.406322,1.000000,0.048067,0.049612,0.034296,0.046260,0.003624,0.010448,0.010103,-0.008222,0.006174,0.332193,0.376510,0.166305,0.030732,-0.000490,-0.061173,0.025246,0.042900,0.041356,0.043719,0.009343,0.052270,0.038603,0.079081,-0.001274,0.043861,0.029586,0.250325,0.001999,0.286128,0.009473,0.043079,0.051797,0.018853,-0.017788
Var21,0.576973,0.063691,0.048067,1.000000,0.997676,0.668110,0.884478,-0.009268,0.031777,0.015128,0.007828,0.003454,0.055701,0.048555,0.014974,0.002046,-0.002046,-0.154138,0.804109,0.755952,0.796984,0.823043,0.006850,0.761418,0.885479,0.002063,0.013002,0.007160,0.013439,0.026805,-0.002210,0.018392,-0.006606,0.017913,0.924531,0.008278,-0.006897
Var22,0.575632,0.067912,0.049612,0.997676,1.000000,0.671668,0.886206,-0.002110,0.032553,0.018594,0.008121,0.003327,0.057694,0.051945,0.015474,0.005395,-0.001879,-0.153779,0.804572,0.760611,0.798926,0.827518,0.009238,0.759649,0.885477,0.002995,0.011956,0.010870,0.016275,0.028098,-0.002092,0.018349,-0.006391,0.023637,0.927281,0.010642,-0.012422
Var24,0.419153,0.065608,0.034296,0.668110,0.671668,1.000000,0.655736,-0.063733,0.010306,-0.117279,-0.006213,-0.003803,0.050771,0.060395,0.012300,-0.052743,-0.005887,-0.232806,0.463775,0.740024,0.770816,0.718283,-0.029814,0.507453,0.503748,0.005954,0.000709,-0.082829,-0.035682,0.027519,-0.000767,-0.001635,0.078825,-0.136438,0.573099,-0.009581,0.000034
Var25,0.505610,0.080325,0.046260,0.884478,0.886206,0.655736,1.000000,-0.043411,0.026328,-0.079803,0.003807,0.004578,0.060549,0.065359,0.017718,-0.053684,-0.002605,-0.218956,0.616011,0.752633,0.683728,0.713805,-0.014320,0.676187,0.705778,0.002959,0.008840,-0.079900,0.063262,0.035243,-0.004482,0.003625,-0.007929,-0.076974,0.790594,0.035778,-0.008085
Var28,-0.016302,-0.022627,0.003624,-0.009268,-0.002110,-0.063733,-0.043411,1.000000,0.082500,0.271758,-0.017167,0.004945,0.020543,-0.076532,0.001373,0.171148,-0.002520,0.233759,0.037218,-0.037872,-0.020860,-0.032680,0.096022,-0.007870,0.048478,-0.006755,0.036959,0.210744,0.162910,0.021895,0.039051,0.077968,0.054290,0.368353,-0.010974,0.122244,-0.005649
Var35,-0.005624,-0.014688,0.010448,0.031777,0.032553,0.010306,0.026328,0.082500,1.000000,0.055666,0.132659,0.004717,0.004377,-0.032000,0.002688,0.010438,0.146098,0.042896,0.028108,0.029604,0.022869,0.023084,-0.012698,0.010163,0.030390,0.001253,0.610661,0.017475,0.019695,0.015601,0.085363,0.022922,0.003713,0.055181,0.030225,0.020547,0.065505
Var38,0.021091,0.020216,0.010103,0.015128,0.018594,-0.117279,-0.079803,0.271758,0.055666,1.000000,0.007649,-0.000849,0.054323,-0.066981,0.001349,0.052642,-0.013292,0.380410,0.123602,-0.073613,-0.019707,-0.061060,0.193054,0.026262,0.157295,-0.012415,-0.000585,0.082638,0.093434,0.016925,0.009965,0.105355,-0.030895,0.690675,-0.025466,0.071866,-0.075818


In [0]:
train=train.dropna(axis=0, how='all',thresh=50,subset=None, inplace=False)
train=train.dropna(axis=1, how='all',thresh=50,subset=None, inplace=False)


train=train.fillna(0)
train.shape

